## 04 - Evaluation

### Import packages and load the data

In [1]:
import pandas as pd
import lightgbm as lgb
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
import math
import matplotlib.pyplot as plt
import numpy as np
import datetime
import plotly.express as px
import pickle

In [2]:
data_input_path = '/Users/szejozsef00/Desktop/MSC/MSC 2. félév/DS Lab I/DSLAB1/data/processed/'
prediction_output_path = '/Users/szejozsef00/Desktop/MSC/MSC 2. félév/DS Lab I/DSLAB1/data/predictions/'

In [3]:
fact_df = pd.read_csv(data_input_path + 'modelling_df.csv',parse_dates=['DATETIME'])
bm_pred = pd.read_csv(prediction_output_path + 'bm_pred_2010_07_01_1_3_d.csv',parse_dates=['DATETIME'])
pred = pd.read_csv(prediction_output_path + 'pred_2010_07_01_0.7.csv',parse_dates=['DATETIME'])

### Merge the data

In [5]:
fact_df = fact_df[(fact_df['DATETIME'].dt.date == datetime.date(2010, 7, 1)) & (fact_df['LOCATION'] < 197)].copy(deep=True)

In [6]:
eval_df = pred.merge(fact_df[['DATETIME','LOCATION','VALUE']], on=['DATETIME','LOCATION'], how='left')
eval_df = eval_df.merge(bm_pred[['DATETIME','LOCATION','bm_1d_prediction']], on=['DATETIME','LOCATION'], how='left')

In [7]:
# Filter out locations where the actual value is 0 all the time
locations_to_keep = eval_df.groupby('LOCATION')['VALUE'].sum().loc[lambda x: x != 0].index
eval_df = eval_df[eval_df['LOCATION'].isin(locations_to_keep)]

eval_df['LOCATION'].nunique()

152

In [8]:
eval_df.head()

,LOCATION,DATETIME,horizont,prediction,lower_bound,upper_bound,VALUE,bm_1d_prediction
0,0,2010-07-01 00:00:00,288,300.749028,250.109792,366.510705,363.47,363.16
1,0,2010-07-01 00:05:00,288,345.093943,295.369055,413.167195,433.95,449.45
2,0,2010-07-01 00:10:00,288,387.951099,329.515273,457.553470,474.54,504.09
3,0,2010-07-01 00:15:00,288,395.202694,326.635271,456.899536,482.43,506.39
4,0,2010-07-01 00:20:00,288,366.147998,330.609473,433.264585,497.15,453.43
...,...,...,...,...,...,...,...,...
56731,196,2010-07-01 23:35:00,288,-162.066571,-174.784203,-143.625715,-142.77,-155.03
56732,196,2010-07-01 23:40:00,288,-163.687044,-175.161466,-144.736385,-144.00,-157.24
56733,196,2010-07-01 23:45:00,288,-159.222623,-174.424410,-140.942031,-140.48,-153.46
56734,196,2010-07-01 23:50:00,288,-164.607705,-175.124435,-144.354345,-141.42,-157.43


In [9]:
# Calculate sMAPE and RMSE by LOCATION for predictions
pred_smape_by_location = eval_df.groupby('LOCATION').apply(
    lambda x: 100 * np.mean(np.abs(x['VALUE'] - x['prediction']) / ((np.abs(x['VALUE']) + np.abs(x['prediction'])) / 2))
).reset_index(name='P_MAPE')

pred_rmse_by_location = eval_df.groupby('LOCATION').apply(
    lambda x: math.sqrt(mean_squared_error(x['VALUE'], x['prediction']))
).reset_index(name='P_RMSE')

pred_stat_df = pd.concat([pred_smape_by_location, pred_rmse_by_location['P_RMSE']], axis=1)

# Calculate sMAPE and RMSE by LOCATION for benchmarks
bm_pred_smape_by_location = eval_df.groupby('LOCATION').apply(
    lambda x: 100 * np.mean(np.abs(x['VALUE'] - x['bm_1d_prediction']) / ((np.abs(x['VALUE']) + np.abs(x['bm_1d_prediction'])) / 2))
).reset_index(name='BM_MAPE')

bm_pred_rmse_by_location = eval_df.groupby('LOCATION').apply(
    lambda x: math.sqrt(mean_squared_error(x['VALUE'], x['bm_1d_prediction']))
).reset_index(name='BM_RMSE')

bm_pred_stat_df = pd.concat([bm_pred_smape_by_location, bm_pred_rmse_by_location['BM_RMSE']], axis=1)

# Combine prediction and benchmark stats
stat_df = pd.concat([pred_stat_df, bm_pred_stat_df['BM_MAPE'], bm_pred_stat_df['BM_RMSE']], axis=1)

# Calculate differences between benchmark and prediction metrics
stat_df['MAPE_DIFF'] = stat_df['BM_MAPE'] - stat_df['P_MAPE']
stat_df['RMSE_DIFF'] = stat_df['BM_RMSE'] - stat_df['P_RMSE']

# Display the final DataFrame
stat_df

,LOCATION,P_MAPE,P_RMSE,BM_MAPE,BM_RMSE,MAPE_DIFF,RMSE_DIFF
0,0,25.193952,111.392841,14.784591,71.123878,-10.409361,-40.268963
1,1,61.544369,68.949176,49.468045,46.419383,-12.076324,-22.529793
2,3,14.651547,97.930823,18.974537,128.436565,4.322989,30.505743
3,4,35.200502,92.636547,25.938407,73.240823,-9.262095,-19.395724
4,5,9.870977,96.082074,13.064560,129.815585,3.193583,33.733511
...,...,...,...,...,...,...,...
147,177,9.450547,54.403150,10.114736,53.638276,0.664189,-0.764873
148,192,2.723018,34.604126,6.955924,110.091152,4.232906,75.487026
149,194,6.013296,48.668033,7.493467,59.029714,1.480171,10.361682
150,195,6.013136,48.300114,7.493467,59.029714,1.480331,10.729600


In [20]:
best_values = stat_df.sort_values(by='MAPE_DIFF', ascending=False).head(10)['LOCATION'].tolist()
worst_values = stat_df.sort_values(by='MAPE_DIFF', ascending=True).head(10)['LOCATION'].tolist()
worst_mape = stat_df.sort_values(by='P_RMSE', ascending=False).head(5)['LOCATION'].tolist()
best_mape = stat_df.sort_values(by='P_RMSE', ascending=True).head(5)['LOCATION'].tolist()

In [11]:
print("MAPE:")
print("P MAPE:",len(stat_df[stat_df['MAPE_DIFF'] > 0]) / len(stat_df)) 
print("BM MAPE:",len(stat_df[stat_df['MAPE_DIFF'] <= 0]) / len(stat_df))
print()
print("RMSE:")
print("P RMSE:",len(stat_df[stat_df['RMSE_DIFF'] > 0]) / len(stat_df)) 
print("BM RMSE:",len(stat_df[stat_df['RMSE_DIFF'] <= 0]) / len(stat_df))

MAPE:
P MAPE: 0.5460526315789473
BM MAPE: 0.45394736842105265

RMSE:
P RMSE: 0.5657894736842105
BM RMSE: 0.4342105263157895


In [12]:
import plotly.graph_objects as go

def plot_predictions(location):
    # Filter the DataFrame for the specific location
    filtered_df = eval_df[eval_df['LOCATION'] == location]

    # Get MAPE and RMSE values for the location
    pred_mape = stat_df.loc[stat_df['LOCATION'] == location, 'P_MAPE'].values[0]
    pred_rmse = stat_df.loc[stat_df['LOCATION'] == location, 'P_RMSE'].values[0]
    bm_mape = stat_df.loc[stat_df['LOCATION'] == location, 'BM_MAPE'].values[0]
    bm_rmse = stat_df.loc[stat_df['LOCATION'] == location, 'BM_RMSE'].values[0]

    # Create the figure
    fig = go.Figure()

    # Add actual values trace
    fig.add_trace(go.Scatter(
        x=filtered_df['DATETIME'],
        y=filtered_df['VALUE'],
        mode='lines',
        name='Actual Value',
        line=dict(color='blue', width=2),
        hovertemplate='Actual: %{y}<br>Date: %{x}'
    ))

    # Add prediction trace
    fig.add_trace(go.Scatter(
        x=filtered_df['DATETIME'],
        y=filtered_df['prediction'],
        mode='lines',
        name='Prediction',
        line=dict(color='orange', width=2),
        hovertemplate='Prediction: %{y}<br>Date: %{x}'
    ))

    # Add benchmark prediction trace
    fig.add_trace(go.Scatter(
        x=filtered_df['DATETIME'],
        y=filtered_df['bm_1d_prediction'],
        mode='lines',
        name='Benchmark Prediction',
        line=dict(color='green', width=2),
        hovertemplate='Benchmark: %{y}<br>Date: %{x}'
    ))

    # Update layout for styling
    fig.update_layout(
        title={
            'text': f"Predictions for Location {location}<br>"
                    f"P_MAPE: {pred_mape:.2f}, P_RMSE: {pred_rmse:.2f}<br>"
                    f"BM_MAPE: {bm_mape:.2f}, BM_RMSE: {bm_rmse:.2f}",
            'y': 0.9,
            'x': 0.5,
            'xanchor': 'center',
            'yanchor': 'top'
        },
        xaxis_title='DateTime',
        yaxis_title='Value',
        legend_title='Legend',
        font=dict(family="Arial, sans-serif", size=12, color="black"),
        # plot_bgcolor="#f9f9f9",
        hovermode='x unified'
    )

    # Show the figure
    fig.show()

In [13]:
for l in best_values:
    plot_predictions(l)

In [14]:
for l in worst_values:
    plot_predictions(l)

In [21]:
for l in best_mape:
    plot_predictions(l)